In [ ]:
from PIL import Image

# ==============================================================================
#  HELPER: UNSLOTH GENERATION WRAPPER
# ==============================================================================
def run_unsloth_generation(model, tokenizer, image, prompt, max_tokens=512):
    """
    Standardizes inference for Unsloth FastVisionModel.
    Unsloth tokenizer handles the image processing directly.
    """
    
    # 1. Prepare the messages using chat template
    messages = [
        {"role": "user", "content": [
            {"type": "image"}, # Placeholder for the image
            {"type": "text", "text": prompt}
        ]}
    ]
    
    # 2. Format text using the tokenizer's template
    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
    )

    # 3. Tokenize & Process Image (Unsloth specific syntax)
    # Unsloth tokenizer accepts ( [image_list], text_prompt, ... )
    inputs = tokenizer(
        [image],
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    # 4. Generate
    # We use 'fast_generate' logic implicit in the model or standard generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        use_cache=True,
        temperature=0.7, # Adjust as needed
        do_sample=False, # Deterministic for scientific accuracy
    )

    # 5. Decode response
    # Skip the input tokens to get only the new response
    response = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:], 
        skip_special_tokens=True
    )
    return response.strip()


# ==============================================================================
#  MAIN FUNCTION: FCC PIPELINE (UNSLOTH VERSION)
# ==============================================================================
def run_fcc_pipeline(model, tokenizer, image_path, user_prompt, cache_dict):
    """
    Args:
        model: Your loaded Unsloth model (with LoRA attached)
        tokenizer: Your loaded Unsloth tokenizer
        image_path: Path to the image file
        user_prompt: The user's query
        cache_dict: A dictionary {} passed from your main loop
    """

    # --- PROMPTS ---
    FCC_STAGE_1_PROMPT = """
    You are analyzing a False Color Composite (FCC) satellite image. FCC colors do NOT represent natural colors—ignore all color-based meaning.
    Describe only what is visually and unambiguously present, in a short output.
    Focus strictly on: geometric shapes, boundaries, edges, patterns, textures.
    Do NOT name objects or land-cover types unless the shape alone makes it certain.
    Do NOT infer, assume, or guess. Output only brief feature-level observations.
    """

    RGB_STAGE_2_TEMPLATE = """
    You are analyzing an RGB image that was converted from an FCC image.
    
    Inputs:
    1. The RGB image
    2. FCC feature summary: {fcc_features}
    
    Task: Combine RGB geometric cues with the FCC feature summary to answer the user's request.
    
    User Request: {user_query}
    
    Rules:
    • No hallucination, no assumptions.
    • Use only evidence supported by geometry + FCC features.
    • Ignore colors if they contradict the FCC summary.
    """

    print(f"   [FCC Pipeline] Processing: {image_path}")
    
    # Load Image once
    try:
        current_image = Image.open(image_path).convert("RGB")
    except Exception as e:
        return f"Error loading image: {e}"

    # --- STAGE 1: FCC FEATURE EXTRACTION (Cached) ---
    if image_path in cache_dict:
        print("   -> ⚡ Using cached FCC features.")
        fcc_features = cache_dict[image_path]
    else:
        print("   -> 🔍 Stage 1: Running FCC Feature Extraction...")
        fcc_features = run_unsloth_generation(
            model, tokenizer, current_image, FCC_STAGE_1_PROMPT, max_tokens=256
        )
        cache_dict[image_path] = fcc_features  # Save to cache

    # --- STAGE 2: RGB SYNTHESIS ---
    # (Note: In your real logic, you might convert the image here. 
    # For now, we reuse the same image object as 'RGB' representation)
    
    print("   -> 🎨 Stage 2: Running RGB Analysis...")
    
    final_prompt = RGB_STAGE_2_TEMPLATE.format(
        fcc_features=fcc_features, 
        user_query=user_prompt
    )
    
    final_response = run_unsloth_generation(
        model, tokenizer, current_image, final_prompt, max_tokens=512
    )
    
    return final_response